In [3]:
import os
import pandas as pd
from openai import OpenAI

In [1]:
import os

api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API key loaded successfully")
else:
    print("API key not found. Please set the environment variable.")

API key loaded successfully


In [4]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [5]:
n100 = pd.read_csv("/Users/yuliaanashkina/Downloads/nasdaq100.csv")
n100_p = pd.read_csv("/Users/yuliaanashkina/Downloads/nasdaq100_price_change.csv")
nasdaq100 = n100.merge(n100_p[["symbol", "ytd"]], on="symbol", how="inner")
nasdaq100.head()

,symbol,name,headQuarter,dateFirstAdded,cik,founded,ytd
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,42.99992
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,68.66902
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,57.22723
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01,17.02062
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01,5.53732


In [9]:
# Loop through the NASDAQ companies
for company in nasdaq100["symbol"]:
    # Create a prompt to enrich nasdaq100 using OpenAI
    prompt = f'''Classify company {company} into one of the following sectors. Answer only with the sector name: Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, Communication, Energy, Consumer Defensive, Real Estate, Financial.
'''
    # Create a request to the completions endpoint
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )
    # Store the output as a variable called sector
    sector = response.choices[0].message.content
    
    # Add the sector for the corresponding company
    nasdaq100.loc[nasdaq100["symbol"] == company, "Sector"] = sector
    
# Count the number of sectors
nasdaq100["Sector"].value_counts()

# Prompt to get stock recommendations
prompt = f'''Provide summary information about Nasdaq-100 stock performance year to date (YTD), recommending the three best sectors               and three or more companies per sector.
            Company data: {nasdaq100} 
'''

# Get the model response
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )

# Store the output as a variable and print the recommendations
stock_recommendations = response.choices[0].message.content
print(stock_recommendations)

The Nasdaq-100 stock performance year to date (YTD) shows that the Technology sector has been performing well, with companies like Apple Inc. (AAPL), Adobe Inc. (ADBE), and Workday, Inc. (WDAY) showing strong YTD growth. The Real Estate sector has also been performing well, with Airbnb (ABNB) leading the way. Additionally, the Financial sector has shown positive growth, with companies like Warner Bros. Discovery (WBD) and Zoom Video Communications (ZM) performing well. Overall, the top three sectors based on YTD performance are Technology, Real Estate, and Financial.
